In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pandas import read_csv
import math
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional
# from keras.layers import LSTM, Dense, Bidirectional, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from tensorflow import keras
from pathlib import Path
from keras.callbacks import ModelCheckpoint
import pandas as pd
from matplotlib import pyplot as plt
import glob
import os
import numpy as np

from collections import Counter

In [ ]:
X = np.load(Path('AllCSVs/X_scaled.npy'))
y = np.load(Path('AllCSVs/y_scaled.npy'))
X.shape, y.shape

In [ ]:
np.mean(abs(y), axis=0)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, shuffle=True)
X_train.shape, y_train.shape

In [ ]:
y[0]

In [ ]:
# # reshape input to be [samples, time steps, features]
# X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
# X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))
# X_train.shape

# LSTM

In [ ]:
# create and fit the LSTM network
n_outputs = X_train.shape[2]

model = Sequential()
model.add(Bidirectional(LSTM(64, input_shape=(None, 4))))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
# model.add(Dense(n_outputs)) 
model.add(Dense(n_outputs))

# model = keras.models.load_model('modelV2.h5')

model.compile(loss='mean_squared_error', optimizer='adam', metrics=[tf.keras.metrics.RootMeanSquaredError()])


mc = ModelCheckpoint('best_model1.h5', monitor='val_loss', mode='min', save_best_only=True)

model.fit(X_train, y_train, 
          epochs=1, 
          batch_size=10000, 
          verbose=1, 
          validation_split=0.2,
          callbacks=[mc])

# model.save('model1_Transformer.h5')
#loss: 847995.6250 - val_loss: 333785.3438
y_pred = model.predict(X_test, batch_size=90000)
mse_1 = ((y_pred - y_test)**2).mean(axis=0)
mse = np.square(np.subtract(y_test,y_pred)).mean() 
rmse = math.sqrt(mse)
print('RMSE')
print('[', math.sqrt(mse_1[0]),math.sqrt(mse_1[1]), math.sqrt(mse_1[2]), math.sqrt(mse_1[3]), ']')
print('Y_test Min:', np.amin(y_test, axis=0),'Y_test Max:', np.amax(y_test, axis=0))

In [ ]:
model.save('model1Final.h5')
y_pred = model.predict(X_test, batch_size=90000)
mse_1 = ((y_pred - y_test)**2).mean(axis=0)
mse = np.square(np.subtract(y_test,y_pred)).mean() 
rmse = math.sqrt(mse)
print('RMSE')
print('[', math.sqrt(mse_1[0]),math.sqrt(mse_1[1]), math.sqrt(mse_1[2]), math.sqrt(mse_1[3]), ']')
print('Y_test Min:', np.amin(y_test, axis=0),'Y_test Max:', np.amax(y_test, axis=0))

In [ ]:
for i in [1,1231,13123,345634,96341,54232,26743,76345,90000]:
    print('Pred Pos: [{:f},{:f}] Spd: [{:f},{:f}]'.format(y_pred[i][0],y_pred[i][1],y_pred[i][2],y_pred[i][3]))
    print('Test Pos: [{:f},{:f}] Spd: [{:f},{:f}]'.format(y_test[i][0],y_test[i][1],y_test[i][2],y_test[i][3]))
    print()

# Transformer

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalMaxPooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.layers import TransformerBlock

# Prepare the data
input_shape = (10,4)  # input shape of each time series sequence
num_classes = X_train.shape[2]

# Define the model
inputs = Input(shape=input_shape)
x = inputs
embedding_dim = 4 
num_heads = 8
ff_dim = 4 

# Time series embedding layer
x = TransformerBlock(embed_dim=embedding_dim, num_heads=num_heads, ff_dim=ff_dim)(x)
x = GlobalMaxPooling1D()(x)
x = Dropout(0.1)(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.1)(x)
outputs = Dense(num_classes)(x)
model = Model(inputs=inputs, outputs=outputs)

n_feats = 4


model.compile(loss='mean_squared_error', optimizer='adam', metrics=[tf.keras.metrics.RootMeanSquaredError()])


mc = ModelCheckpoint('model1_Transformer.h5', monitor='val_loss', mode='min', save_best_only=True)

# for i in range(0,100):
# print('Epoch:', i)
# X_train, y_train = get_data(X_attacker_train, X_normal_train)
# y_train = tf.keras.utils.to_categorical(y_train)


model.fit(X_train, y_train, 
          epochs=1, 
          batch_size=9000, 
          validation_split=0.1,
#           callbacks=[mc]
         )
model.summary()
# model.save('model1_Transformer.h5')


In [ ]:
model.fit(X_train, y_train, 
          epochs=100, 
          batch_size=2000, 
#           validation_split=0.1,
#           callbacks=[mc]
         )

In [ ]:
y_pred = model.predict(X_test[0:1])
y_pred

In [ ]:
X_test[0:1].shape

In [ ]:
y_pred = model.predict(X_test, batch_size=90000)
mse_1 = ((y_pred - y_test)**2).mean(axis=0)
mse = np.square(np.subtract(y_test,y_pred)).mean() 
rmse = math.sqrt(mse)
print('RMSE')
print('[', math.sqrt(mse_1[0]),math.sqrt(mse_1[1]), math.sqrt(mse_1[2]), math.sqrt(mse_1[3]), ']')
print('Y_test Min:', np.amin(y_test, axis=0),'Y_test Max:', np.amax(y_test, axis=0))

In [ ]:
for i in [1,1231,13123,345634,96341,54232,26743,76345,90000]:
    print('Pred Pos: [{:f},{:f}] Spd: [{:f},{:f}]'.format(y_pred[i][0],y_pred[i][1],y_pred[i][2],y_pred[i][3]))
    print('Test Pos: [{:f},{:f}] Spd: [{:f},{:f}]'.format(y_test[i][0],y_test[i][1],y_test[i][2],y_test[i][3]))
    print()

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalMaxPooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers

# from tensorflow.keras.layers import TransformerBlock
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)
#     def get_config(self):

#         config = super().get_config().copy()
#         config.update({
#             'num_heads': self.num_heads,
#             'embed_dim': self.embed_dim,
#             'ff_dim': self.ff_dim,
#         })
#         return config
#https://stackoverflow.com/questions/58678836/notimplementederror-layers-with-arguments-in-init-must-override-get-conf

# MAKING CSVS

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pandas import read_csv
import math
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional
# from keras.layers import LSTM, Dense, Bidirectional, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from tensorflow import keras
from pathlib import Path
from keras.callbacks import ModelCheckpoint
import pandas as pd
from matplotlib import pyplot as plt
import glob
import os
import numpy as np

from collections import Counter
model = keras.models.load_model('model1Final.h5')
# y_pred = model.predict(X_test, batch_size=90000)
# mse_1 = ((y_pred - y_test)**2).mean(axis=0)
# mse = np.square(np.subtract(y_test,y_pred)).mean() 
# rmse = math.sqrt(mse)
# print('RMSE')
# print('[', math.sqrt(mse_1[0]),math.sqrt(mse_1[1]), math.sqrt(mse_1[2]), math.sqrt(mse_1[3]), ']')
# print('Y_test Min:', np.amin(y_test, axis=0),'Y_test Max:', np.amax(y_test, axis=0))

In [ ]:
start_folder = 'AllCSVs'
csvs = sorted(glob.glob('{}/*.csv'.format(start_folder)))
df_list_og = []

for c in csvs:
    df = pd.read_csv(c)
    for i in np.unique(df.sender):
        df_list_og.append(df[df.sender.isin([i])].copy())
    #     df[df.ID.isin([subs[0]])&df.Vigil.isin(vigils[0])].copy()
    
print(len(df_list_og))
# df_list[0]

In [ ]:
len(df_list_og)

In [ ]:
df_list = df_list_og.copy()
# df_attacker = []
# df_normal = []
window_size = 10
overlap_size = 9

# for count, j in enumerate(df_list):
for count in range(20712, len(df_list)):#46012
    j = df_list[count]
    if j.attackerType.any() == 0:
        j = j.iloc[:,[4,5,6,7]]
        for start_idx in range(0, len(j)-window_size, window_size - overlap_size):
            end_idx = start_idx + window_size
            df_chunk = j.iloc[start_idx:end_idx]
            y_test = np.expand_dims(np.asarray(df_chunk), axis=0)

            pred_idx = j.iloc[end_idx].name
            y_pred = model.predict(y_test, verbose=0)
            
            df_list[count].loc[pred_idx, 'pos0_Pred'] = y_pred[0][0]
            df_list[count].loc[pred_idx, 'pos1_Pred'] = y_pred[0][1]
            df_list[count].loc[pred_idx, 'spd0_Pred'] = y_pred[0][2]
            df_list[count].loc[pred_idx, 'spd1_Pred'] = y_pred[0][3]
#             display(df_chunk)
           # df_normal.append(df_chunk)
        df_list[count] = df_list[count].drop(df_list[count].index[-1])
        df_list[count].to_csv('CSVsPred/Pred_Car_{}_normal.csv'.format(count)) 
        print('CSVsPred/Pred_Car_{}_normal.csv'.format(count))
    elif j.attackerType.any() == 1:
#         print('Attack')
        j = j.iloc[:,[4,5,6,7]]
#         for start_idx in range(0, len(j)-window_size, window_size - overlap_size):
#             end_idx = start_idx + window_size
#             df_chunk = j.iloc[start_idx:end_idx]
#             y_test = np.expand_dims(np.asarray(df_chunk), axis=0)

#             pred_idx = j.iloc[end_idx].name
#             y_pred = model.predict(y_test, verbose=0)
            
#             df_list[count].loc[pred_idx, 'pos0_Pred'] = y_pred[0][0]
#             df_list[count].loc[pred_idx, 'pos1_Pred'] = y_pred[0][1]
#             df_list[count].loc[pred_idx, 'spd0_Pred'] = y_pred[0][2]
#             df_list[count].loc[pred_idx, 'spd1_Pred'] = y_pred[0][3]
# #             display(df_chunk)
# #             df_attacker.append(df_chunk)
#         df_list[count] = df_list[count].drop(df_list[count].index[-1])
#         print('CSVsPred/Pred_Car_{}_attacker.csv'.format(count))
#         df_list[count].to_csv('CSVsPred/Pred_Car_{}_attacker.csv'.format(count)) 

# df_normal = np.asarray(df_normal)
# df_attacker = np.asarray(df_attacker)

# df_normal[0]